In [1]:
from keras.models import Sequential , Model
from keras.layers import Dense ,  BatchNormalization , Reshape , Input , Flatten
from keras.layers import Conv2D , MaxPool2D , Conv2DTranspose , UpSampling2D , ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Activation
from keras.layers import Dropout

from keras.optimizers import Adam

from keras.datasets import mnist

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os

import matplotlib as plt
import numpy as np

import gc

%matplotlib inline

In [3]:
WIDTH = 96
HEIGHT = 96
CHANNEL = 3

LATENT_DIM = 100 #latent variable z sample from normal distribution

BATCH_SIZE = 64
EPOCHS = 10

PATH = 'faces/'

#生成多少个图像 长*宽
ROW = 5
COL = 5


In [13]:

load_index = 0

images_name = os.listdir(PATH)

IMAGES_COUNT = len(images_name)


In [5]:
'''
(X_train , y_train),(X_test , y_test) = mnist.load_data()
X_train = X_train/127.5-1
X_train = np.expand_dims(X_train , 3)
'''

In [6]:
'''
def load_mnist():
    return X_train[np.random.randint(0, X_train.shape[0], BATCH_SIZE)]
    
def write_image_mnist(epoch):
    
    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))
    generated_image = generator_i.predict(noise)
    generated_image = generated_image*0.5+0.5
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    
    count=0
    
    for i in range(ROW):
        for j in range(COL):
            axes[i][j].imshow(generated_image[count,:,:,0] , cmap = 'gray')
            axes[i][j].axis('off')
            count += 1
            
    fig.savefig('mnist_dcgan/No.%d.png' % epoch)
    plt.pyplot.close()

'''

In [11]:

def load_image(batch_size = BATCH_SIZE):
    global load_index
    
    images = []
    
    for i in range(batch_size):
        images.append(plt.image.imread(PATH + images_name[(load_index + i) % IMAGES_COUNT]))
    
    load_index += batch_size
    
    return np.array(images)/127.5-1

def write_image(epoch):
    
    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))
    generated_image = generator_i.predict(noise)
    generated_image = (generated_image+1)*127.5
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    
    count=0
    
    for i in range(ROW):
        for j in range(COL):
            axes[i][j].imshow(generated_image[count])
            axes[i][j].axis('off')
            count += 1
            
    fig.savefig('generated_faces_dcgan/No.%d.png' % epoch)
    plt.pyplot.close()
    
    #plt.image.imsave('images/'+str(epoch)+'.jpg')


In [5]:
def generator():
    #sample from noise z
    model = Sequential(name='generator')
    
    #cartoon 图像使用 96*96*3
    model.add(Dense(128 * 6 * 6, activation="relu", input_shape=(LATENT_DIM,)))
    model.add(Reshape((6, 6, 128)))
    model.add(UpSampling2D()) #12
    
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D()) #24
    
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D()) #48

    model.add(Conv2D(32, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D()) #96
    
    model.add(Conv2D(CHANNEL, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))
    #
    model.summary()
    
    noise = Input(shape=(LATENT_DIM , ) , name='input1')
    image = model(noise)
    
    return Model(noise , image , name='generator_Model')

In [6]:
def discriminator():
    #input a image to discriminate real or fake
    model = Sequential(name='discriminator')
    
    model.add(Conv2D(filters=32 , kernel_size=(3,3) , strides=(2,2) , padding='same' , input_shape=(WIDTH , HEIGHT , CHANNEL) , name='conv1'))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.25))
    
    #model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(filters=64 , kernel_size=(3,3) , strides=(2,2) , padding='same' , name='conv2'))
    model.add(ZeroPadding2D(padding=((0,1) , (0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters=128 , kernel_size=(3,3) , strides=(2,2) , padding='same' , name='conv3'))
    model.add(BatchNormalization(momentum=0.8))  
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.25))

    
    model.add(Conv2D(filters=256 , kernel_size=(3,3) , strides=(1,1) , name='conv4'))
    model.add(BatchNormalization(momentum=0.8))  
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    
    model.add(Dense(units=1 , activation='sigmoid' , name='dense16'))
    
    model.summary()
    
    image = Input(shape=(WIDTH , HEIGHT , CHANNEL) , name='input1')
    validity = model(image)
    
    return Model(image , validity , name='discriminator_Model')

In [7]:
def combined_model(generator_i , discriminator_i):
    #生成器和判别器组合成整体
    z = Input(shape=(LATENT_DIM , ) , name='z')
    
    image = generator_i(z)
    
    #print(image.shape)
    
    discriminator_i.trainable = False
    validity = discriminator_i(image)
    
    return Model(z , validity , name='combined_model')

In [8]:
adam = Adam(lr = 0.0002 , beta_1=0.5)

In [9]:
discriminator_i = discriminator()
discriminator_i.compile(optimizer=adam , loss='binary_crossentropy' , metrics=['accuracy'])

generator_i = generator()

combined_model_i = combined_model(generator_i , discriminator_i)


combined_model_i.compile(optimizer=adam , loss='binary_crossentropy')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 48, 48, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 48, 48, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 48, 32)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 24, 24, 64)        18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 25, 25, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 25, 25, 64)        256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 25, 25, 64)        0         
__________

In [ ]:
real_labels = np.ones(shape=(BATCH_SIZE , 1)) #真实样本label为1
fake_labels = np.zeros(shape=(BATCH_SIZE , 1)) #假样本label为0

for i in range(10000):
    noise = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))

    real_image = load_image()
    #real_image = load_mnist()
    #训练判别器
    fake_image = generator_i.predict(noise)

    real_loss = discriminator_i.train_on_batch(real_image , real_labels)
    fake_loss = discriminator_i.train_on_batch(fake_image , fake_labels)

    loss = np.add(real_loss , fake_loss)/2

    #训练生成器
    noise2 = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))
    generator_loss = combined_model_i.train_on_batch(noise2 , real_labels)
    
    print('epoch:%d loss:%f accu:%f gene_loss:%f' % (i , loss[0] , loss[1] , generator_loss))
    
    if i % 100 == 0:
        write_image(i)
        #write_image_mnist(i)
    
write_image(999)
#write_image_mnist(999)


C:\Anaconda3\lib\site-packages\keras\engine\training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 loss:1.446787 accu:0.320312 gene_loss:0.455436
epoch:1 loss:0.731037 accu:0.585938 gene_loss:1.398017
epoch:2 loss:0.594655 accu:0.718750 gene_loss:2.022584
epoch:3 loss:0.446518 accu:0.820312 gene_loss:1.710653
epoch:4 loss:0.424779 accu:0.812500 gene_loss:2.188331
epoch:5 loss:0.405232 accu:0.835938 gene_loss:2.421367
epoch:6 loss:0.213117 accu:0.937500 gene_loss:3.141307
epoch:7 loss:0.314316 accu:0.882812 gene_loss:2.807391
epoch:8 loss:0.371971 accu:0.820312 gene_loss:2.587402
epoch:9 loss:0.268522 accu:0.882812 gene_loss:3.041912
epoch:10 loss:0.220139 accu:0.945312 gene_loss:3.102090
epoch:11 loss:0.349809 accu:0.867188 gene_loss:2.574595
epoch:12 loss:0.375242 accu:0.773438 gene_loss:3.276208
epoch:13 loss:0.251004 accu:0.906250 gene_loss:3.350988
epoch:14 loss:0.400142 accu:0.812500 gene_loss:3.600238
epoch:15 loss:0.332151 accu:0.859375 gene_loss:2.285674
epoch:16 loss:0.269793 accu:0.882812 gene_loss:1.184132
epoch:17 loss:0.155001 accu:0.945312 gene_loss:0.612140
ep

epoch:146 loss:0.488054 accu:0.789062 gene_loss:2.171204
epoch:147 loss:0.219134 accu:0.914062 gene_loss:1.223020
epoch:148 loss:0.190821 accu:0.953125 gene_loss:1.427864
epoch:149 loss:0.150101 accu:0.945312 gene_loss:1.784628
epoch:150 loss:0.158958 accu:0.937500 gene_loss:0.559735
epoch:151 loss:0.166552 accu:0.929688 gene_loss:0.200103
epoch:152 loss:0.134319 accu:0.953125 gene_loss:0.651035
epoch:153 loss:0.073959 accu:0.992188 gene_loss:1.053143
epoch:154 loss:0.344693 accu:0.820312 gene_loss:1.316391
epoch:155 loss:0.544638 accu:0.765625 gene_loss:2.347795
epoch:156 loss:1.121507 accu:0.406250 gene_loss:3.766684
epoch:157 loss:0.625963 accu:0.726562 gene_loss:3.298679
epoch:158 loss:0.450189 accu:0.757812 gene_loss:3.820457
epoch:159 loss:0.800742 accu:0.687500 gene_loss:0.846168
epoch:160 loss:0.170257 accu:0.937500 gene_loss:1.763065
epoch:161 loss:0.069584 accu:0.984375 gene_loss:2.030299
epoch:162 loss:0.139376 accu:0.960938 gene_loss:1.245799
epoch:163 loss:0.621095 accu:0.

epoch:290 loss:0.127368 accu:0.960938 gene_loss:0.134852
epoch:291 loss:0.021329 accu:1.000000 gene_loss:0.186122
epoch:292 loss:0.096693 accu:0.976562 gene_loss:0.412163
epoch:293 loss:0.057874 accu:0.992188 gene_loss:0.409344
epoch:294 loss:0.149925 accu:0.953125 gene_loss:0.797484
epoch:295 loss:0.284250 accu:0.867188 gene_loss:1.390096
epoch:296 loss:0.260966 accu:0.898438 gene_loss:2.346503
epoch:297 loss:0.755359 accu:0.640625 gene_loss:3.133653
epoch:298 loss:0.813296 accu:0.648438 gene_loss:1.331667
epoch:299 loss:0.226096 accu:0.929688 gene_loss:0.828487
epoch:300 loss:0.246491 accu:0.921875 gene_loss:0.931587
epoch:301 loss:0.376088 accu:0.812500 gene_loss:0.735767
epoch:302 loss:0.101868 accu:0.968750 gene_loss:0.641072
epoch:303 loss:0.179165 accu:0.953125 gene_loss:0.464328
epoch:304 loss:0.273243 accu:0.867188 gene_loss:1.023208
epoch:305 loss:0.340312 accu:0.867188 gene_loss:0.779819
epoch:306 loss:0.601379 accu:0.734375 gene_loss:1.014649
epoch:307 loss:0.574082 accu:0.

In [70]:
real_labels.shape

(64, 1)

In [53]:
discriminator_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 28, 28, 1)         0         
_________________________________________________________________
discriminator (Sequential)   (None, 1)                 533505    
Total params: 533,505
Trainable params: 0
Non-trainable params: 533,505
_________________________________________________________________


In [54]:
generator_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 100)               0         
_________________________________________________________________
generator (Sequential)       (None, 28, 28, 1)         1097744   
Total params: 1,097,744
Trainable params: 1,095,184
Non-trainable params: 2,560
_________________________________________________________________


In [39]:
combined_model_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z (InputLayer)               (None, 100)               0         
_________________________________________________________________
generator_Model (Model)      (None, 96, 96, 3)         29029120  
_________________________________________________________________
discriminator_Model (Model)  (None, 1)                 14320641  
Total params: 43,349,761
Trainable params: 29,025,536
Non-trainable params: 14,324,225
_________________________________________________________________


In [ ]:
gc.collect()

In [14]:
modeli = Sequential()

modeli.add(Dense(128 * 7 * 7, activation="relu", input_shape=(LATENT_DIM,)))
modeli.add(Reshape((7, 7, 128)))
modeli.add(UpSampling2D())
modeli.add(Conv2D(128, kernel_size=3, padding="same"))
modeli.add(BatchNormalization(momentum=0.8))
modeli.add(Activation("relu"))
modeli.add(UpSampling2D())
modeli.add(Conv2D(64, kernel_size=3, padding="same"))
modeli.add(BatchNormalization(momentum=0.8))
modeli.add(Activation("relu"))
modeli.add(Conv2D(CHANNEL, kernel_size=3, padding="same"))
modeli.add(Activation("tanh"))

In [15]:
modeli.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_3 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 28, 28, 128)       0         
__________

In [41]:
32*400

12800

In [ ]:
gc.collect()